# Knowledge Graph RAG - Version Ultra Simple

Version minimaliste du pipeline Knowledge Graph + RAG sans robustesse ni gestion d'erreur.

In [ ]:
# %pip install streamlit
# %pip install yfiles-jupyter-graphs-for-neo4j
# %pip install neo4j 
# %pip install python-dotenv 
# %pip install langchain 
# %pip install langchain-community
# %pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


: 

## 1. Imports et Configuration

In [28]:
# Imports essentiels
import os
import json
import uuid
import glob
from dotenv import load_dotenv

# LangChain
from langchain_community.graphs import Neo4jGraph
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [29]:
# Import de yFiles pour Neo4j
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
from neo4j import GraphDatabase

In [30]:
# Configuration
load_dotenv('.env', override=True)

# Neo4j
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

# OpenAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [31]:
# Connexion Neo4j
kg = Neo4jGraph(
    url=NEO4J_URI, 
    username=NEO4J_USERNAME, 
    password=NEO4J_PASSWORD, 
    database=NEO4J_DATABASE
)

print("✅ Configuration terminée")

✅ Configuration terminée


## 2. Traitement des PDFs

In [32]:
# Traitement des PDFs
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
pdf_files = glob.glob("PDFs/*.pdf")
all_chunks = []

for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pages = loader.load()
    chunks = text_splitter.split_documents(pages)
    
    for i, chunk in enumerate(chunks):
        chunk_data = {
            'id': str(uuid.uuid4()),
            'filename': os.path.basename(pdf_file),
            'chunk_index': i,
            'text': chunk.page_content
        }
        all_chunks.append(chunk_data)

print(f"✅ {len(all_chunks)} chunks créés")

✅ 349 chunks créés


## 3. Génération d'Embeddings

## 4. Création du Knowledge Graph

In [33]:
# Générer embeddings
embeddings_service = OpenAIEmbeddings(model="text-embedding-3-small")

# Embeddings pour chunks
chunk_texts = [chunk['text'] for chunk in all_chunks]
chunk_embeddings = embeddings_service.embed_documents(chunk_texts)

for i, chunk in enumerate(all_chunks):
    chunk['embedding'] = chunk_embeddings[i]

print(f"✅ Embeddings générés pour {len(all_chunks)} chunks")

✅ Embeddings générés pour 349 chunks


## 5. Création du Knowledge Graph

In [34]:
# Nettoyer la base
kg.query("MATCH (n) DETACH DELETE n")

# Créer nœuds Chunk seulement
for chunk in all_chunks:
    kg.query("""
    CREATE (c:Chunk {
        id: $id,
        text: $text,
        filename: $filename,
        chunk_index: $chunk_index
    })
    """, params=chunk)

print("✅ Nœuds Chunk créés")

✅ Nœuds Chunk créés


In [35]:
# Créer relations NEXT seulement
kg.query("""
MATCH (c1:Chunk), (c2:Chunk)
WHERE c1.filename = c2.filename 
  AND c1.chunk_index + 1 = c2.chunk_index
CREATE (c1)-[:NEXT]->(c2)
""")

print("✅ Relations NEXT créées")

✅ Relations NEXT créées


## 4.5. Ajout des Embeddings dans Neo4j

In [36]:
# Créer index vectoriel pour les embeddings
kg.query("""
CREATE VECTOR INDEX simple_chunks IF NOT EXISTS
FOR (c:Chunk) ON (c.textEmbedding) 
OPTIONS { indexConfig: {
    `vector.dimensions`: 1536,
    `vector.similarity_function`: 'cosine'    
}}
""")

print("✅ Index vectoriel créé")

✅ Index vectoriel créé


In [37]:
# Sauvegarder les embeddings dans Neo4j
# Méthode 1: En utilisant les embeddings déjà calculés en mémoire
print("Sauvegarde des embeddings dans Neo4j...")

for chunk in all_chunks:
    kg.query("""
    MATCH (c:Chunk {id: $chunk_id})
    CALL db.create.setNodeVectorProperty(c, "textEmbedding", $embedding)
    RETURN c.id as saved_id
    """, params={
        'chunk_id': chunk['id'], 
        'embedding': chunk['embedding']
    })

print(f"✅ Embeddings sauvegardés pour {len(all_chunks)} chunks dans Neo4j")

Sauvegarde des embeddings dans Neo4j...
✅ Embeddings sauvegardés pour 349 chunks dans Neo4j
✅ Embeddings sauvegardés pour 349 chunks dans Neo4j


In [38]:
# Diagnostic: Vérifier les index existants
print("📋 Index existants dans Neo4j:")
indexes = kg.query("SHOW INDEXES")
for index in indexes:
    print(f"  - {index.get('name', 'N/A')}: {index.get('type', 'N/A')} - État: {index.get('state', 'N/A')}")

print(f"\n📊 Nombre total d'index: {len(indexes)}")

# Vérifier spécifiquement les index vectoriels
vector_indexes = [idx for idx in indexes if 'vector' in str(idx.get('type', '')).lower()]
print(f"📐 Index vectoriels trouvés: {len(vector_indexes)}")

# Vérifier si les chunks ont des embeddings
chunks_with_embeddings = kg.query("""
MATCH (c:Chunk) 
WHERE c.textEmbedding IS NOT NULL
RETURN count(c) as count
""")
print(f"🔢 Chunks avec embeddings: {chunks_with_embeddings[0]['count'] if chunks_with_embeddings else 0}")

📋 Index existants dans Neo4j:
  - GrahRAG: VECTOR - État: ONLINE
  - index_343aff4e: LOOKUP - État: ONLINE
  - index_f7700477: LOOKUP - État: ONLINE

📊 Nombre total d'index: 3
📐 Index vectoriels trouvés: 1
  - GrahRAG: VECTOR - État: ONLINE
  - index_343aff4e: LOOKUP - État: ONLINE
  - index_f7700477: LOOKUP - État: ONLINE

📊 Nombre total d'index: 3
📐 Index vectoriels trouvés: 1
🔢 Chunks avec embeddings: 349
🔢 Chunks avec embeddings: 349


In [39]:
# Solution: Utiliser l'index existant ou en créer un nouveau
print("🔧 Gestion de l'index vectoriel...")

# Vérifier les index existants
indexes = kg.query("SHOW INDEXES")
vector_indexes = [idx for idx in indexes if 'vector' in str(idx.get('type', '')).lower()]

if vector_indexes:
    existing_index = vector_indexes[0]['name']
    print(f"  ✅ Index vectoriel existant trouvé: '{existing_index}'")
    VECTOR_INDEX_NAME = existing_index
else:
    # Créer un nouvel index avec un nom unique
    import time
    timestamp = int(time.time())
    VECTOR_INDEX_NAME = f"chunks_vector_{timestamp}"
    
    try:
        kg.query(f"""
        CREATE VECTOR INDEX {VECTOR_INDEX_NAME}
        FOR (c:Chunk) ON (c.textEmbedding) 
        OPTIONS {{ indexConfig: {{
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
        }}}}
        """)
        print(f"  ✅ Nouvel index vectoriel créé: '{VECTOR_INDEX_NAME}'")
    except Exception as e:
        print(f"  ❌ Erreur lors de la création: {e}")
        VECTOR_INDEX_NAME = "GrahRAG"  # Fallback

print(f"📋 Index vectoriel à utiliser: '{VECTOR_INDEX_NAME}'")

🔧 Gestion de l'index vectoriel...
  ✅ Index vectoriel existant trouvé: 'GrahRAG'
📋 Index vectoriel à utiliser: 'GrahRAG'


## 5. Recherche Vectorielle avec Neo4j

## 6. RAG Simple

In [40]:
# Fonction de recherche vectorielle utilisant Neo4j
def neo4j_vector_search(question, top_k=3):
    """Recherche vectorielle utilisant l'index Neo4j"""
    # Utiliser l'index vectoriel disponible
    index_name = VECTOR_INDEX_NAME if 'VECTOR_INDEX_NAME' in globals() else 'GrahRAG'
    
    # Générer embedding de la question
    question_embedding = embeddings_service.embed_query(question)
    
    try:
        # Recherche vectorielle dans Neo4j
        vector_search_query = f"""
        CALL db.index.vector.queryNodes('{index_name}', $top_k, $question_embedding) 
        YIELD node, score
        RETURN score, node.text AS text, node.filename AS source, node.id AS chunk_id
        ORDER BY score DESC
        """
        
        results = kg.query(vector_search_query, params={
            'question_embedding': question_embedding,
            'top_k': top_k
        })
        
        return results
    except Exception as e:
        print(f"❌ Erreur lors de la recherche vectorielle avec l'index '{index_name}': {e}")
        return []

# Test de la recherche vectorielle
results = neo4j_vector_search("Luxembourg")
print("🔍 Résultats de recherche vectorielle:")
for i, result in enumerate(results):
    print(f"{i+1}. Score: {result['score']:.3f}")
    print(f"   {result['text'][:150]}...")
    print(f"   Source: {result['source']}")
    print()

🔍 Résultats de recherche vectorielle:
1. Score: 0.762
   passant de la 19e place du classement 2019 de la Commission européenne à la 3e place en 2022. Cette 3
e 
place a été confirmée en 2023 et 2024 . En 20...
   Source: rapport-activite-2024-fin.pdf

2. Score: 0.745
   nationaux afin de permettre aux citoyens, aux entreprises et aux administrations de profiter pleinement 
du marché unique numérique. Il vise à créer l...
   Source: rapport-activite-2024-fin.pdf

3. Score: 0.745
   général de la société.  
Dans le cadre du E -Leaders, le Luxembourg participe également à des groupes de travail thématiques 
divers dont ceux traitan...
   Source: rapport-activite-2024-fin.pdf



## 7. Visualisation Basic

In [41]:
# Fonction RAG avec recherche vectorielle Neo4j
def ask_question_vectorial(question):
    # Rechercher contexte avec la recherche vectorielle
    results = neo4j_vector_search(question, top_k=3)
    context = "\n\n".join([r['text'] for r in results])
    
    # Générer réponse
    prompt = f"""
Contexte trouvé par recherche vectorielle:
{context}

Question: {question}

Réponds en français basé uniquement sur le contexte:
"""
    
    llm = ChatOpenAI(temperature=0)
    response = llm.invoke(prompt)
    return response.content, results

# Test du RAG vectoriel
answer, sources = ask_question_vectorial("Qu'est-ce que Luxembourg?")
print("🤖 Réponse RAG Vectorielle:")
print(answer)
print("\n📚 Sources utilisées:")
for i, source in enumerate(sources):
    print(f"{i+1}. Score: {source['score']:.3f} - {source['source']}")
print("\n" + "="*50)

🤖 Réponse RAG Vectorielle:
Le Luxembourg est un pays membre fondateur de l'EUROPEUM-EDIC, un consortium visant à déployer et étendre l'Infrastructure européenne de services blockchain. Le Luxembourg participe activement à la gouvernance de l'EUROPEUM-EDIC et aux réunions des groupes de travail du consortium. Le pays a également enregistré une progression significative dans le domaine des services publics numériques, passant de la 19e place en 2019 à la 3e place en 2022, confirmée en 2023 et 2024. Le Luxembourg vise à promouvoir des services numériques homogènes dans l'UE et à permettre aux citoyens, aux entreprises et aux administrations de profiter pleinement du marché unique numérique.

📚 Sources utilisées:
1. Score: 0.741 - rapport-activite-2024-fin.pdf
2. Score: 0.737 - rapport-activite-2024-fin.pdf
3. Score: 0.736 - rapport-activite-2024-fin.pdf



## 8. Interface Interactive de Questions

In [44]:
# Interface interactive avec widgets
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import time

# Installation automatique si nécessaire
try:
    import ipywidgets
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ipywidgets"])
    import ipywidgets as widgets

print("📦 Widgets chargés avec succès!")

📦 Widgets chargés avec succès!


In [45]:
# Créer l'interface interactive
class RAGChatInterface:
    def __init__(self):
        # Widgets pour l'interface
        self.question_input = widgets.Textarea(
            value='',
            placeholder='Posez votre question ici...',
            description='Question:',
            layout=widgets.Layout(width='100%', height='80px')
        )
        
        self.ask_button = widgets.Button(
            description='🔍 Poser la Question',
            button_style='primary',
            layout=widgets.Layout(width='200px')
        )
        
        self.clear_button = widgets.Button(
            description='🗑️ Effacer',
            button_style='warning',
            layout=widgets.Layout(width='120px')
        )
        
        self.top_k_slider = widgets.IntSlider(
            value=3,
            min=1,
            max=10,
            step=1,
            description='Nb sources:',
            layout=widgets.Layout(width='300px')
        )
        
        self.output_area = widgets.Output()
        
        # Connecter les événements
        self.ask_button.on_click(self.on_ask_clicked)
        self.clear_button.on_click(self.on_clear_clicked)
        
        # Questions prédéfinies
        self.preset_questions = [
            "Qu'est-ce que Luxembourg?",
            "Quels sont les projets numériques au Luxembourg?",
            "Comment le Luxembourg utilise-t-il l'IA?",
            "Quelles sont les initiatives blockchain au Luxembourg?",
            "Quel est le rôle du CTIE?"
        ]
        
        self.preset_dropdown = widgets.Dropdown(
            options=['Choisir une question...'] + self.preset_questions,
            description='Exemples:',
            layout=widgets.Layout(width='400px')
        )
        self.preset_dropdown.observe(self.on_preset_selected, names='value')
    
    def on_preset_selected(self, change):
        if change['new'] != 'Choisir une question...':
            self.question_input.value = change['new']
            self.preset_dropdown.value = 'Choisir une question...'
    
    def on_ask_clicked(self, button):
        question = self.question_input.value.strip()
        if not question:
            with self.output_area:
                print("⚠️ Veuillez saisir une question")
            return
        
        with self.output_area:
            clear_output()
            print(f"🤔 Question: {question}")
            print("🔍 Recherche en cours...")
            
            start_time = time.time()
            
            try:
                # Effectuer la recherche RAG
                answer, sources = ask_question_vectorial(question)
                search_time = time.time() - start_time
                
                print(f"⏱️ Temps de recherche: {search_time:.2f}s")
                print("\\n" + "="*60)
                print("🤖 RÉPONSE:")
                print("="*60)
                print(answer)
                
                print("\\n" + "="*60)
                print("📚 SOURCES UTILISÉES:")
                print("="*60)
                
                for i, source in enumerate(sources[:self.top_k_slider.value]):
                    print(f"\\n{i+1}. 📄 {source['source']}")
                    print(f"   📊 Score de pertinence: {source['score']:.3f}")
                    print(f"   📝 Extrait: {source['text'][:200]}...")
                
            except Exception as e:
                print(f"❌ Erreur: {e}")
    
    def on_clear_clicked(self, button):
        with self.output_area:
            clear_output()
        self.question_input.value = ''
    
    def display(self):
        # Layout de l'interface
        controls = widgets.VBox([
            widgets.HTML("<h3>🎯 Interface de Questions RAG</h3>"),
            self.preset_dropdown,
            self.question_input,
            widgets.HBox([self.ask_button, self.clear_button, self.top_k_slider]),
            widgets.HTML("<hr>")
        ])
        
        full_interface = widgets.VBox([controls, self.output_area])
        display(full_interface)

# Créer et afficher l'interface
chat_interface = RAGChatInterface()
chat_interface.display()

In [ ]:
# Interface alternative simple (sans widgets)
def simple_chat():
    """Interface de chat simple pour environnements sans widgets"""
    print("🎯 Interface Simple de Questions RAG")
    print("="*50)
    print("📝 Questions d'exemple:")
    examples = [
        "Qu'est-ce que Luxembourg?",
        "Quels sont les projets numériques au Luxembourg?",
        "Comment le Luxembourg utilise-t-il l'IA?",
        "Quelles sont les initiatives blockchain au Luxembourg?",
        "Quel est le rôle du CTIE?"
    ]
    
    for i, example in enumerate(examples, 1):
        print(f"{i}. {example}")
    
    print("\\n" + "="*50)
    
    while True:
        try:
            question = input("\\n🤔 Votre question (ou 'quit' pour quitter): ").strip()
            
            if question.lower() in ['quit', 'exit', 'q']:
                print("👋 Au revoir!")
                break
            
            if not question:
                print("⚠️ Veuillez saisir une question")
                continue
            
            print(f"\\n🔍 Recherche pour: {question}")
            print("-" * 40)
            
            start_time = time.time()
            answer, sources = ask_question_vectorial(question)
            search_time = time.time() - start_time
            
            print(f"⏱️ Temps: {search_time:.2f}s")
            print("\\n🤖 RÉPONSE:")
            print(answer)
            
            print("\\n📚 SOURCES:")
            for i, source in enumerate(sources, 1):
                print(f"{i}. {source['source']} (Score: {source['score']:.3f})")
            
        except KeyboardInterrupt:
            print("\\n👋 Au revoir!")
            break
        except Exception as e:
            print(f"❌ Erreur: {e}")

# Décommenter la ligne suivante pour utiliser l'interface simple
# simple_chat()

### 📖 Guide d'utilisation de l'interface

**🎯 Interface Interactive (ci-dessus) :**
1. **Questions prédéfinies** : Utilisez le menu déroulant pour sélectionner une question d'exemple
2. **Question personnalisée** : Tapez votre question dans la zone de texte
3. **Nombre de sources** : Ajustez le curseur pour choisir combien de sources afficher (1-10)
4. **Poser la question** : Cliquez sur le bouton bleu pour obtenir une réponse
5. **Effacer** : Utilisez le bouton orange pour nettoyer l'affichage

**🔍 Fonctionnalités :**
- ⏱️ **Temps de réponse** affiché
- 📊 **Score de pertinence** pour chaque source
- 📝 **Extraits** des documents sources
- 🤖 **Réponse contextuelle** générée par l'IA

**💡 Conseils :**
- Posez des questions **spécifiques** sur le Luxembourg, le numérique, l'IA, la blockchain
- Les questions en **français** donnent de meilleurs résultats
- Utilisez des **mots-clés précis** pour des réponses plus pertinentes

In [42]:
# Statistiques du graphe avec embeddings
stats = kg.query("""
MATCH (n) 
RETURN labels(n)[0] as type, count(n) as count
""")

print("📊 Statistiques du Knowledge Graph avec embeddings:")
for stat in stats:
    print(f"   {stat['type']}: {stat['count']}")

# Relations
relations = kg.query("""
MATCH ()-[r]->() 
RETURN type(r) as type, count(r) as count
""")

print("\n🔗 Relations:")
for rel in relations:
    print(f"   {rel['type']}: {rel['count']}")

# Vérifier les embeddings
embedding_check = kg.query("""
MATCH (c:Chunk) 
WHERE c.textEmbedding IS NOT NULL
RETURN count(c) as chunks_with_embeddings
""")

print(f"\n📐 Chunks avec embeddings: {embedding_check[0]['chunks_with_embeddings']}")

# Index vectoriel
indexes = kg.query("SHOW INDEXES")
vector_indexes = [idx for idx in indexes if 'vector' in idx.get('type', '').lower()]
print(f"🔍 Index vectoriels: {len(vector_indexes)}")

print("\n✅ Knowledge Graph RAG Vectoriel Terminé!")
print("📝 Pipeline: PDF → Chunks → Embeddings → Neo4j → Recherche Vectorielle → RAG")

📊 Statistiques du Knowledge Graph avec embeddings:
   Chunk: 349

🔗 Relations:
   NEXT: 347

🔗 Relations:
   NEXT: 347

📐 Chunks avec embeddings: 349

📐 Chunks avec embeddings: 349
🔍 Index vectoriels: 1

✅ Knowledge Graph RAG Vectoriel Terminé!
📝 Pipeline: PDF → Chunks → Embeddings → Neo4j → Recherche Vectorielle → RAG
🔍 Index vectoriels: 1

✅ Knowledge Graph RAG Vectoriel Terminé!
📝 Pipeline: PDF → Chunks → Embeddings → Neo4j → Recherche Vectorielle → RAG


## 9. Statistiques et Visualisation

In [43]:
driver = GraphDatabase.driver(
    uri = NEO4J_URI, 
    auth = (NEO4J_USERNAME, NEO4J_PASSWORD), 
    database = NEO4J_DATABASE
    )
g = Neo4jGraphWidget(driver)

g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t")


GraphWidget(layout=Layout(height='800px', width='100%'))